
# 과거 딥러닝의 근본적 문제점

* 과적합
* 기울기 소멸
* 성능 하락

# 문제 발생 원인

* 과적합 - 훈련 데이터를 과도하게 학습하여, 검증/실제 데이터에 대한 오차가 증가함.

* 기울기 소멸 - 은닉층이 많아지다 보면, 오차에 대한 계산도 많아져 오차가 크게 감소, 결국은 이에 대한 기울기가 소멸(0에 수렴)하여 학습이 불가해짐.

* 성능 하락 - 경사 하강법은 오차가 가장 작게 되는 지점을 찾는데, 이 과정에서 시간 낭비, 적합점 탐색 불가, 발산 등의 성능이 하락하는 문제들이 발생.



---



# 과적합 문제 해결 방안

* 과적합 - 드롭아웃

In [ ]:
# 과적합 문제 발생

import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# MNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 데이터 전처리
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0

# 과적합을 위해 훈련 데이터 증강(의도)
x_train = np.concatenate((x_train, x_train[:1000]))
y_train = np.concatenate((y_train, y_train[:1000]))

# 모델 구성
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(784,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

In [ ]:
# 이 코드를 추가

from tensorflow.keras.layers import Dropout

# 모델 구성
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(784,)))
model.add(Dropout(0.5))  # 드롭아웃 층 추가
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # 드롭아웃 층 추가
model.add(Dense(10, activation='softmax'))

드롭아웃은 입력된 데이터에 대해 학습하는 과정 중 학습에 사용되는 일부 노드(뉴런)들을 학습에서 제외시켜 과적합을 막을 수 있는 방법이다. 과적합 문제 뿐 아니라 모델이 더욱 향상된 일반화 성능을 가질 수 있게 해 준다.

In [ ]:
# 드롭아웃 구성 예제
class DropoutModel(torch.nn.Module):
    def __init__(self):
        super(DropoutModel, self).__init__()
        self.layer1 = torch.nn.Linear(784, 1200)
        self.dropout1 = torch.nn.Dropout(0.5) # 50%의 노드를 무작위로 선택하여 사용하지 않겠다는 의미

        self.layer2 = torch.nn.Linear(1200, 1200)
        self.dropout2 = torch.nn.Dropout(0.5) # "

        self.layer3 = torch.nn.Linear(1200, 10)

    def forward(self, x):
        x = F.relu( self.layer1(x) )
        x = self.dropout1(x)

        x = F.relu( self.layer2(x) )
        x = self.dropout2(x)

        return self.layer3(x)



---



# 기울기 소멸 문제 해결 방안

* 기울기 소멸 - ReLU

In [ ]:
# 시그모이드 함수의 정의

import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
# PyToch

import torch
import torch.nn.functional as F

torch.sigmoid(x)

은닉층이 많은 신경망에서, 역전파 과정 중 오차가 0에 수렴하여 가중치에 대한 업데이트가 이루어지지 않는 현상이다.

주 원인은 활성화 함수로 시그모이드 함수가 사용되기 때문인데, 시그모이드 함수는 0~1 사이의 값을 출력하며, 만약 작은 값이 시그모이드 함수를 사용한 역전파 과정에서 계속 곱해지면 기울기가 점차 감소한다.

그렇기에 시그모이드 함수 대신 렐루 함수를 활성화 함수로 사용하면 이 문제가 해결된다.

In [ ]:
# ReLU 함수의 정의
def relu(x):
    return max(0, x)

In [ ]:
# PyTorch

import torch
import torch.nn.functional as F

F.relu(x)



---



# 성능 하락 문제 해결 방안

* 성능 하락 - 확률적 경사 하강법 / 미니 배치 경사 하강법

In [ ]:
# 일반적인 경사 하강법 알고리즘

def gradient_descent(X, y, learning_rate, num_iterations):
    num_samples, num_features = X.shape
    weights = np.zeros(num_features)
    bias = 0

    for _ in range(num_iterations):
        # 예측 계산
        y_pred = np.dot(X, weights) + bias

        # 오차 계산
        error = y_pred - y

        # 가중치와 편향의 업데이트
        weights -= (learning_rate / num_samples) * np.dot(X.T, error)
        bias -= (learning_rate / num_samples) * np.sum(error)

    return weights, bias

경사 하강법은 강력한 알고리즘이지만 지역 최솟값 수렴, 계산 시간 급증, 발산 등의 성능이 하락하는 문제가 발생할 가능성이 있다. 이를 해결하기 위해 미니 배치 경사 하강법, 확률적 경사 하강법 등을 사용한다.

## 배치 경사 하강법
배치 경사 하강법은 Batch, 말 그대로 경사 하강법에 대해 일괄처리를 한다는 뜻이다. 전체 데이터셋에 대한 오류를 계산한 뒤 기울기를 한 번만 계산하여 모델의 파라미터를 업데이트한다.

In [ ]:
# 배치 경사 하강법 알고리즘

import numpy as np

def batch_gradient_descent(X, y, learning_rate, num_iterations):
    num_samples, num_features = X.shape
    weights = np.zeros(num_features)
    bias = 0

    for _ in range(num_iterations):
        # 예측 계산
        y_pred = np.dot(X, weights) + bias

        # 오차 계산
        error = y_pred - y

        # 가중치와 편향의 업데이트
        weights -= (learning_rate / num_samples) * np.dot(X.T, error)
        bias -= (learning_rate / num_samples) * np.sum(error)

    return weights, bias


### 미니 배치 경사 하강법
배치 경사 하강법과 개념적으로는 비슷하지만, 일괄처리하는 묶음을 여러 개로 쪼갠다는 차이점이 있다. 즉, 전체 데이터셋을 미니 배치 여러 개로 나누고 각각의 미니 배치에 대한 일괄 처리 계산 후 그것들의 평균 기울기를 사용해 모델의 파라미터를 업데이트한다.

In [ ]:
# 미니 배치 경사 하강법 알고리즘

import numpy as np

def mini_batch_gradient_descent(X, y, learning_rate, batch_size, num_iterations):
    num_samples, num_features = X.shape
    weights = np.zeros(num_features)
    bias = 0

    for _ in range(num_iterations):
        # 배치 샘플 선택
        indices = np.random.choice(num_samples, batch_size, replace=False)
        X_batch = X[indices]
        y_batch = y[indices]

        # 예측 계산
        y_pred = np.dot(X_batch, weights) + bias

        # 오차 계산
        error = y_pred - y_batch

        # 가중치와 편향의 업데이트
        weights -= (learning_rate / batch_size) * np.dot(X_batch.T, error)
        bias -= (learning_rate / batch_size) * np.sum(error)

    return weights, bias


In [ ]:
# 미니 배치 경사 하강법 구성 예제

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [ [1,2,3], [4,5,6], [7,8,9] ]
        self.y_data = [ [12], [18], [11] ]

        def __len__(self):
            return len(self.x_data)

        def __getitem__(self, idx):
            x = torch.FloatTensor(self.x_data[idx])
            y = torch.FloatTensor(self.y_data[idx])
            return x, y

dataset = CustomDataset()
dataloader = DataLoader( dataset, batch_size=2, shuffle=True )

## 확률적 경사 하강법
확률적 경사 하강법은 임의로 선택한 데이터에 대해 기울기를 계산하고 가중치와 편향을 업데이트한다.

In [ ]:
# 확률적 경사 하강법 알고리즘

import numpy as np

def stochastic_gradient_descent(X, y, learning_rate, num_iterations):
    num_samples, num_features = X.shape
    weights = np.zeros(num_features)
    bias = 0

    for _ in range(num_iterations):
        # 무작위로 훈련 샘플 선택
        index = np.random.randint(num_samples)
        x = X[index]
        label = y[index]

        # 예측 계산
        y_pred = np.dot(x, weights) + bias

        # 오차 계산
        error = y_pred - label

        # 가중치와 편향의 업데이트
        weights -= learning_rate * error * x
        bias -= learning_rate * error

    return weights, bias


확률적 경사 하강법은 무작위로 하나의 훈련 샘플을 선택해 연산을 진행하기 때문에 파라미터의 변경 폭이 불안정하나 속도가 비교적 빠르다.